IMPORTING LIBRARIES

In [26]:
import pandas as pd
import os
import re
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/giangtrongnguyen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/giangtrongnguyen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

INPUTTING THE FILES

This set of code aims to encode the open-end responses from the "data" sheet by using the codebook of each question, in this case is "CodebookQ3" and "CodebookQ5"

In [27]:
input_path=r"/Users/giangtrongnguyen/Python/Portfolio/OE_Encoding/SantaClara_Survey.xlsx"
other_code=98

codebook1 = pd.read_excel(input_path, sheet_name='CodebookQ3')
codebook2 = pd.read_excel(input_path, sheet_name='CodebookQ5')
# codebook3 = pd.read_excel(input_path, sheet_name='codebookQ47')
# codebook4 = pd.read_excel(input_path, sheet_name='Codebook')
# codebook5 = pd.read_excel(input_path, sheet_name='Codebook')
# codebook6 = pd.read_excel(input_path, sheet_name='Codebook')

codebooks={'codebook1': codebook1,
 'codebook2': codebook2,
# 'codebook3': codebook3,
 # 'codebook4': codebook4,
 # 'codebook5': codebook5,
 # 'codebook6': codebook6,
 }
data = pd.read_excel(input_path, sheet_name='data')

Here is how the "data" sheet looks like

In [28]:
data

,id,token,Q3,Q5
0,1,2482ae9ae6,Crime has increased significantly in my neighb...,I support Trump's stance on immigration and bo...
1,2,79979fd5c5,Limited mental health services. It's expensive,He improved the job market significantly durin...
2,3,b928bd5d25,Expensive healthcare!!!,His tax reforms helped small businesses thrive.
3,4,5e50ef0bde,Housing price is stupid,I don’t like how he handled the COVID-19 pande...
4,5,c18365046b,You can not manage waste,Trump was too confrontational with foreign all...
...,...,...,...,...
95,96,a1bb8f08da,Public transport is unreliable,He mobilized a significant political movement.
96,97,b194f43891,Water shortages??? Why you turn them off so often,Trump often contradicted his own team publicly.
97,98,56eac40abe,Crime has increased significantly,He turned press briefings into spectacles.
98,99,af5f006fa9,Lack of job,His legacy is one of polarization and upheaval.


Here is how the "CodebookQ3" and "CodebookQ5" look like
Q3. What do you think are critical issue the City of Santa Clara is facing?
Q5. What topics come to mind when you think of Donald Trump?

In [29]:
#codebookQ3
codebook1

,Code,Description
0,1,Housing affordability
1,2,Traffic and congestion
2,3,Public transportation issues
3,4,Homelessness
4,5,Environment/ Green space/ Air quality
5,6,Crime and safety
6,7,Development/ Overdevelopment
7,8,Water/ Waste/ Utility concerns
8,9,Education system/ Schooling
9,10,Healthcare accessibility/Mental health service


In [43]:
#codebookQ5
codebook2

,Code,Description
0,1,Economy / Jobs / Tax reform / Deregulation/ Ma...
1,2,Border control / Immigration / National securi...
2,3,Trade deals / Rural focus / Conservative values
3,4,Neutral / Mixed views / Balanced opinion / Pol...
4,5,COVID handling / Padenmic/ Isolationism / Publ...
5,6,Aggressive rhetoric / Inappropriate behavior /...
6,7,Social media / Tweets / Press briefings / Ente...
7,8,Misinformation / Conspiracy / Election denial ...
8,9,Undermining institutions / Disregard norms / C...
9,10,Climate policy / Religion in politics / Health...


TEXT CLEANING FUNCTION. This function is to:
- Split the response into a list of separate words
- Remove the stop words in the response
- Remove the special character in the response
- Combine the separate words into a cleaned sentence

In [31]:
def clean_stop_word(x):
   if pd.isna(x):
      return ""
   a = re.sub(r"[^a-zA-Z\s]", "", str(x)).lower()
   stop_words = set(stopwords.words('english'))
   word_tokens = word_tokenize(a)
   filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words and re.search(r'\w', w)]
   return ' '.join(filtered_sentence)

TEXT STEMMING FUNCTION.
This function is to stem the text, convert the vocabularies in the same word family into the same format. For example: "educate", "education", "educational" will be all converted to "educ"

In [32]:
# STEMMING FUNCTION
stemmer = PorterStemmer()
def stem_text(text):
   if pd.isna(text):
      return set()
   words = word_tokenize(str(text).lower())
   return [stemmer.stem(word) for word in words]


CREATING CODED COLUMNS, APPLYING TEXT CLEANING & STEMMING FUNCTIONS

In [33]:
oe_questions=[]
cloned_oe_questions=[]

for i, col_name in enumerate(list(data.columns)):
   if i>=2:
      oe_questions.append(col_name)
      data[f"{col_name}x"] = data[col_name]
      cloned_oe_questions.append(f"{col_name}x")
      data[f"{col_name}x"]= data[f"{col_name}x"].apply(clean_stop_word)

      for j in range(3,0,-1):
         add_col=f"{col_name}x_Coded{j}"
         data.insert(data.columns.get_loc(col_name)+1, add_col, np.nan)

JUST SOME DISCLAIMER TO INTERACT WITH THE USERS

In [34]:
print()
print('Hi there! Thank you for choosing me! Please bare in mind that I myself can not replace human work completely for OE coding. You MUST recheck the final result before submit the official data')
print('The process may take some time, let have your eyes rested a bit!!!\n')


Hi there! Thank you for choosing me! Please bare in mind that I myself can not replace human work completely for OE coding. You MUST recheck the final result before submit the official data
The process may take some time, let have your eyes rested a bit!!!



In [35]:
# CODING OE
for idx, a in enumerate(cloned_oe_questions): # This loop goes through each question that need coding
    print(f'Working on {a.replace("x","")}...') 
    codebook = codebooks[str('codebook'+str(idx+1))] 
    for i in range(len(data)): # This loop goes through each response
        answer_words = stem_text(data.loc[i, a]) 
        used_codes = set()

        for m in answer_words: # This loop goes through each word in the responses
            for j in range(len(codebook)): # This loop goes through the codebook
                for col in[str(a+'_Coded1'), str(a+'_Coded2'), str(a+'_Coded3')]: # This loop goes through coded column 1 - 2 - 3
                    
                    code = codebook.loc[j, 'Code'] # Get the code from the codebook

                    if not answer_words: # If the response is empty, skip to the next iteration
                        data.loc[i, col]=np.nan
                        continue
                   
                    if (codebook.loc[j, 'Code'] == other_code) or (code in used_codes) or (not pd.isna(data.loc[i, col])): # Skip to the next iteration if the code is 'other_code', or if the code has been used, or if the column is already filled
                        continue

                    # APPLYING CLEANING & STEMMING FUNCTION FOR THE CODEBOOKS
                    keywords = [clean_stop_word(kw.lower().strip()) for kw in str(codebook.loc[j, 'Description']).split("/")] 
                    keywords = [i for kw in keywords for i in word_tokenize(kw)]
                    keyword_stems = [stemmer.stem(kw) for kw in keywords]
 
                    if m in keyword_stems: # If the word in the response matches the word of the current code in the codebook, encode it
                        data.loc[i, col] = codebook.loc[j, 'Code']
                        used_codes.add(code)
                        break

        if pd.isna(data.loc[i, str(a+'_Coded1')]) and answer_words: # If the first coded column is still empty and the response is not empty, fill it with 'other_code'
            data.loc[i, str(a+'_Coded1')] = other_code
    print(f'Finish {a.replace("x","")}!\n') 

Working on Q3...
Finish Q3!

Working on Q5...
Finish Q5!



FORMATING & EXPORT FINAL DATA


In [36]:
data.drop(columns=cloned_oe_questions, inplace=True)

for i in data.columns:
    data.rename(columns=lambda i: i.replace("x",""), inplace=True)

folder_path = os.path.dirname(input_path)
output_path = os.path.join(folder_path, "OE-coded result.xlsx")

data.to_excel(output_path, index=False)

print(f"The coding process is done, you now may check your file at {output_path}")

The coding process is done, you now may check your file at /Users/giangtrongnguyen/Python/Portfolio/OE_Encoding/OE-coded result.xlsx


The encoded data will look like this

In [37]:
data.head(20)

,id,token,Q3,Q3_Coded1,Q3_Coded2,Q3_Coded3,Q5,Q5_Coded1,Q5_Coded2,Q5_Coded3
0,1,2482ae9ae6,Crime has increased significantly in my neighb...,6.0,NaN,NaN,I support Trump's stance on immigration and bo...,2.0,NaN,NaN
1,2,79979fd5c5,Limited mental health services. It's expensive,10.0,NaN,NaN,He improved the job market significantly durin...,1.0,NaN,NaN
2,3,b928bd5d25,Expensive healthcare!!!,10.0,NaN,NaN,His tax reforms helped small businesses thrive.,1.0,NaN,NaN
3,4,5e50ef0bde,Housing price is stupid,1.0,NaN,NaN,I don’t like how he handled the COVID-19 pande...,5.0,NaN,NaN
4,5,c18365046b,You can not manage waste,8.0,NaN,NaN,Trump was too confrontational with foreign all...,98.0,NaN,NaN
5,6,f1957267db,Gentrification pushing out locals and homeless,4.0,NaN,NaN,His speech style is too aggressive for a presi...,6.0,NaN,NaN
6,7,977041adcb,Housing is damn too expensive,1.0,NaN,NaN,He resonates with working-class Americans.,98.0,NaN,NaN
7,8,c95444f125,Homeless people on the street,4.0,NaN,NaN,His environmental policies were harmful.,10.0,NaN,NaN
8,9,354f4032b6,Overcrowded schools,9.0,NaN,NaN,"He speaks plainly, which appeals to many.",98.0,NaN,NaN
9,10,6b4303a90a,Noise pollution from highways the traffic are ...,2.0,NaN,NaN,Trump made the Supreme Court more conservative.,3.0,NaN,NaN


In [39]:
#Codebook for Q3
codebook1

,Code,Description
0,1,Housing affordability
1,2,Traffic and congestion
2,3,Public transportation issues
3,4,Homelessness
4,5,Environment/ Green space/ Air quality
5,6,Crime and safety
6,7,Development/ Overdevelopment
7,8,Water/ Waste/ Utility concerns
8,9,Education system/ Schooling
9,10,Healthcare accessibility/Mental health service


In [40]:
#Codebook for Q45
codebook2

,Code,Description
0,1,Economy / Jobs / Tax reform / Deregulation/ Ma...
1,2,Border control / Immigration / National securi...
2,3,Trade deals / Rural focus / Conservative values
3,4,Neutral / Mixed views / Balanced opinion / Pol...
4,5,COVID handling / Padenmic/ Isolationism / Publ...
5,6,Aggressive rhetoric / Inappropriate behavior /...
6,7,Social media / Tweets / Press briefings / Ente...
7,8,Misinformation / Conspiracy / Election denial ...
8,9,Undermining institutions / Disregard norms / C...
9,10,Climate policy / Religion in politics / Health...


This set of code can help us shorten the process of OE coding, however, can not completely replace human work as these task require human intuition to define the tone and mood of the response. It can work well with responses that contain specific words in the codebook but with the responses that mention general ideas or relevant words but not the same word family in the codebook, it can not encode them properly

For example, in Q3, there are some responses can be encoded as code 5 ("Environment/ Green space/ Air quality") but this set of code could not recognize it

In [41]:
data.loc[data['Q3_Coded1']==other_code, 'id':'Q3_Coded3']

,id,token,Q3,Q3_Coded1,Q3_Coded2,Q3_Coded3
29,30,1dce9308aa,Noise pollution from highways,98.0,NaN,NaN
32,33,42e4d2d841,Climate change impact is visible you see,98.0,NaN,NaN
34,35,75c14253fe,Climate change impact is visible. They are doi...,98.0,NaN,NaN
59,60,51838f5555,Gentrification and I'm poor,98.0,NaN,NaN
70,71,d217ab58af,Climate change impact is visible for sure,98.0,NaN,NaN
81,82,7760fd6a8e,Climate change impact is visible nobody do any...,98.0,NaN,NaN
86,87,16382b52ed,Climate change impact is for sure visible,98.0,NaN,NaN
89,90,d48c82ece7,Climate change impact is visible,98.0,NaN,NaN


In Q5, a lot of responses could be encoded but the responses and the codebook do not share the same word family, therefore they are skipped

In [42]:
data.loc[data['Q5_Coded1']==other_code, ['id','token', 'Q5', 'Q5_Coded1', 'Q5_Coded2', 'Q5_Coded3']]

,id,token,Q5,Q5_Coded1,Q5_Coded2,Q5_Coded3
4,5,c18365046b,Trump was too confrontational with foreign all...,98.0,NaN,NaN
6,7,977041adcb,He resonates with working-class Americans.,98.0,NaN,NaN
8,9,354f4032b6,"He speaks plainly, which appeals to many.",98.0,NaN,NaN
12,13,fabaaa738f,Trump disrespected democratic processes.,98.0,NaN,NaN
17,18,81a2443a29,Trump’s approach divided the country.,98.0,NaN,NaN
23,24,81a0c47b20,He misused executive powers.,98.0,NaN,NaN
25,26,fefd9825d3,He cut unnecessary regulations.,98.0,NaN,NaN
32,33,42e4d2d841,He reduced global cooperation.,98.0,NaN,NaN
34,35,75c14253fe,He appealed to nationalist sentiments.,98.0,NaN,NaN
36,37,3d35c3472e,"He was loyal to his base, not all Americans.",98.0,NaN,NaN


With this Stemming method, the codebooks need to cover as many keywords that appear in the responses as many as possible. Human quality check is also demanded to minimize the mistakes. However, comparing to manual encoding, it could help to save a plenty of time